In [1]:
import os
import pandas as pd
import torch
import tempfile
from transformers import set_seed, Trainer, TrainingArguments
from tsfm_public import TimeSeriesPreprocessor, get_datasets
from tsfm_public.toolkit.get_model import get_model
from tsfm_public.toolkit.visualization import plot_predictions
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# Set seed for reproducibility
SEED = 42
set_seed(SEED)

# Path to your dataset
file_path = "system_metrics_preprocessed.csv"

In [7]:
timestamp_column = "timestamp"
features = ["system_time", "chrony_last_offset", "chrony_frequency_drift_ppm",
            "chrony_residual_freq_ppm", "chrony_skew", "cpu_temp", "cpu_freq",
            "network_rtt", "power", "cpu_load"]
target = ["chrony_system_time_offset"]

df = pd.read_csv(
    file_path,
    parse_dates=[timestamp_column],
)

# Drop NaN values
df.dropna(subset=features + target, inplace=True)

In [8]:
# Define Split Configuration (80% train, 10% valid, 10% test)
train_size = int(len(df) * 0.8)
valid_size = int(len(df) * 0.1)
split_config = {
    "train": [0, train_size],
    "valid": [train_size, train_size + valid_size],
    "test": [train_size + valid_size, len(df)],
}

# Column Specifications for Preprocessing
column_specifiers = {
    "timestamp_column": timestamp_column,
    "id_columns": [],
    "target_columns": target,
    "control_columns": features,
}

In [9]:
# Model Path (Use the IBM Granite TTM R2 Model)
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r2"

# Context and Prediction Lengths
CONTEXT_LENGTH = 512
PREDICTION_LENGTH = 10  # Adjustable

In [ ]:
tsp = TimeSeriesPreprocessor(
    **column_specifiers,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    scaling=True,
    encode_categorical=False,
    scaler_type="standard",
)

In [ ]:
zeroshot_model = get_model(
    TTM_MODEL_PATH,
    context_length=CONTEXT_LENGTH,
    prediction_length=PREDICTION_LENGTH,
    freq_prefix_tuning=False,
    freq=None,
    prefer_l1_loss=False,
    prefer_longer_context=True,
)


In [ ]:
dset_train, dset_valid, dset_test = get_datasets(
    tsp, df, split_config, use_frequency_token=zeroshot_model.config.resolution_prefix_tuning
)

In [ ]:
temp_dir = tempfile.mkdtemp()
zeroshot_trainer = Trainer(
    model=zeroshot_model,
    args=TrainingArguments(
        output_dir=temp_dir,
        per_device_eval_batch_size=64,  # Batch size for evaluation
        seed=SEED,
        report_to="none",
    ),
)

In [ ]:
zeroshot_output = zeroshot_trainer.evaluate(dset_test)
print(zeroshot_output)